In [2]:
# Import required libraries and dependencies

import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline
import time
import base64
import hmac
import hashlib

# Load .env enviroment variables into the notebook
load_dotenv()

# Get the API key from the environment variable and store as Python variable
kucoin_api_key = os.getenv("KUCOIN_API_KEY")
kucoin_secret_key = os.getenv("KUCOIN_SECRET_KEY")
kucoin_passphrase = os.getenv("KUCOIN_PASSPHRASE")


#pull user account filled spot trade orders data with kucoin api call format
api_key = kucoin_api_key
api_secret = kucoin_secret_key
api_passphrase = kucoin_passphrase
url = 'https://api.kucoin.com/api/v1/fills?currentPage=1&pageSize=500'
now = int(time.time() * 1000)
str_to_sign = str(now) + 'GET' + '/api/v1/fills?currentPage=1&pageSize=500'
signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": "2"
}

kucoin_response = requests.request('get', url, headers=headers)
#print(kucoin_response.status_code)
kucoin_order_fills = kucoin_response.json()
#print(json.dumps(kucoin_order_fills, indent=2, sort_keys=True))

#use the kucoin response data to pull specific data, organize and convert to workable DataFrame
items = kucoin_order_fills["data"]["items"][:]
trading_pair = []
price = []
order_type = []
quantity = []
cost_usdt = []
fee_usdt = []     

index = 0
for index in range(len(items)):
    for key in items[index]:
        if key == "symbol":
            trading_pair.append(items[index][key])
        if key == "price":
            price.append(items[index][key])
        if key == "side":
            order_type.append(items[index][key])
        if key == "size":
            quantity.append(items[index][key])  
        if key == "funds":
            cost_usdt.append(items[index][key])
        if key == "fee":
            fee_usdt.append(items[index][key])
        

trading_pair_dict = {"Trading Pair":trading_pair}
trading_pair_df = pd.DataFrame(trading_pair_dict)

price_dict = {"Price":price}
price_df = pd.DataFrame(price_dict)

order_type_dict = {"Order Type":order_type}
order_type_df = pd.DataFrame(order_type_dict)

quantity_dict = {"Quantity":quantity}
quantity_df = pd.DataFrame(quantity_dict)

cost_usdt_dict = {"Cost USDT":cost_usdt}
cost_usdt_df = pd.DataFrame(cost_usdt_dict)

fee_usdt_dict = {"Fee USDT":fee_usdt}
fee_usdt_df = pd.DataFrame(fee_usdt_dict)


# Create complete DataFrame 
kucoin_data_past_seven_days_df = trading_pair_df\
.join(price_df, how='left')\
.join(order_type_df, how='left')\
.join(quantity_df, how='left')\
.join(cost_usdt_df, how='left')\
.join(fee_usdt_df, how='left')

# Display complete DataFrame
#display(kucoin_data_past_seven_days_df)
#display(kucoin_data_past_seven_days_df.describe())


def count_buys_sells(df):
    buy_sell_count = df.groupby(["Trading Pair", "Order Type"]).agg(count_col=pd.NamedAgg(column="Order Type", aggfunc="count"))
    buy_sell_count_df = pd.DataFrame(buy_sell_count)
    display(buy_sell_count)
    #print(f"You bought {buy_sell_count_df["count_col"]}")
    #display(buy_sell_count_df)

count_buys_sells_df = count_buys_sells(kucoin_data_past_seven_days_df)
#display(count_buys_sells_df.iloc["count_col"])


count_col
Trading Pair  Order Type           
BLOK-USDT     buy                 7
DYDX-USDT     buy                 7
              sell                7
HAKA-USDT     buy                31
HOTCROSS-USDT buy                 1
KEEP-USDT     buy                 2
LSS-USDT      buy                 3
RMRK-USDT     buy                12
              sell                5
SKU-USDT      sell               31